In [1]:
import pandas as pd
import os
import logging
import requests
from bs4 import BeautifulSoup
import random
import re
import urllib.parse
import json

In [2]:
BASE_HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}

In [3]:
base_url = 'https://www.greatschools.org/new-york/new-york/schools/?view=table'

In [4]:
def get_num_pages(url):
    response = requests.get(url, headers=BASE_HEADERS)
    jsonStr = re.search(r'gon.search=(.*?);', response.text).group(1)
    jsonObj = json.loads(jsonStr)
    total_pages = jsonObj['totalPages']
    response.close()
    print('Connection closed')
    return total_pages

In [5]:
num_pages = get_num_pages(base_url)
num_pages

Connection closed


46

In [6]:
def scrape_one_page(url):
    records = []
    response = requests.get(url, headers=BASE_HEADERS)
    jsonStr = re.search(r'gon.search=(.*?);', response.text).group(1)
    jsonObj = json.loads(jsonStr)
    for school in jsonObj['schools']:
        try:
            records.append({ 
                        'School': school['name'], 
                        'Rating': school['rating'], 
                        'Rating scale': school['ratingScale'],
                        'Address': school['address']['street1'], 
                        'Type': school['schoolType'],
                        'Grades': school['gradeLevels'],
                        'Total Students Enrolled': school['enrollment'],
                        'Students per teacher': school['studentsPerTeacher'], 
                        'District': school['districtName'],
                        'URL': school['links']['profile']
                        })
        except KeyError: 
            records.append({ 
                        'School': school['name'], 
                        'Rating': school['rating'], 
                        'Rating scale': school['ratingScale'],
                        'Address': school['address']['street1'], 
                        'Type': school['schoolType'],
                        'Grades': school['gradeLevels'],
                        'Total Students Enrolled': school['enrollment'],
                        'Students per teacher': 'N/A', 
                        'District': school['districtName'],
                        'URL': school['links']['profile']
                        })
    response.close()
    print('Connection closed')
    return records

In [10]:
for page in range (1, num_pages + 1):
    df = pd.DataFrame()
    url = 'https://www.greatschools.org/new-york/new-york/schools/?page={}&view=table'.format(page)
    records = scrape_one_page(url)
    df = pd.DataFrame(records)
    df.to_csv('./greatschools.csv', sep=';', encoding='UTF-8', index=False, mode='a')
    time.sleep(random.randint(1, 50))